In [1]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to  https://aritter.github.io/CS-7650-sp22/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)
# Contributors: Xurui Zhang (Spring 2022)

# Project #2: Named Entity Recognition

In this assignment, you will implement a bidirectional LSTM-CNN-CRF for sequence labeling, following [this paper by Xuezhe Ma and Ed Hovy](https://www.aclweb.org/anthology/P16-1101.pdf), on the CoNLL named entity recognition dataset.  Before starting the assignment, we recommend reading the Ma and Hovy paper.

First, let's import some libraries and make sure the runtime has access to a GPU.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Sun Mar 13 22:13:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download the Data

Run the following code to download the English part of the CoNLL 2003 dataset, the evaluation script and pre-filtered GloVe embeddings we are providing for this data.

In [3]:
#CoNLL 2003 data
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testb
!cat eng.train | awk '{print $1 "\t" $4}' > train
!cat eng.testa | awk '{print $1 "\t" $4}' > dev
!cat eng.testb | awk '{print $1 "\t" $4}' > test

#Evaluation Script
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl

#Pre-filtered GloVe embeddings
!wget https://raw.githubusercontent.com/aritter/aritter.github.io/master/files/glove.840B.300d.conll_filtered.txt

--2022-03-13 22:13:25--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.03s   

2022-03-13 22:13:25 (123 MB/s) - ‘eng.train’ saved [3283420/3283420]

--2022-03-13 22:13:25--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

## CoNLL Data Format

Run the following cell to see a sample of the data in CoNLL format.  As you can see, each line in the file represents a word and its labeled named entity tag in BIO format.  A blank line is used to seperate sentences.

In [4]:
!head -n 20 train

-DOCSTART-	O
	
EU	I-ORG
rejects	O
German	I-MISC
call	O
to	O
boycott	O
British	I-MISC
lamb	O
.	O
	
Peter	I-PER
Blackburn	I-PER
	
BRUSSELS	I-LOC
1996-08-22	O
	
The	O
European	I-ORG


## Reading in the Data

Below we proivide a bit of code to read in data in the CoNLL format.  This also reads in the filtered GloVe embeddings, to save you some effort - we will discuss this more later.

In [5]:
#Read in the training data
def read_conll_format(filename):
    (words, tags, currentSent, currentTags) = ([],[],['-START-'],['START'])
    for line in open(filename).readlines():
        line = line.strip()
        #print(line)
        if line == "":
            currentSent.append('-END-')
            currentTags.append('END')
            words.append(currentSent)
            tags.append(currentTags)
            (currentSent, currentTags) = (['-START-'], ['START'])
        else:
            (word, tag) = line.split()
            currentSent.append(word)
            currentTags.append(tag)
    return (words, tags)

def sentences2char(sentences):
    return [[['start'] + [c for c in w] + ['end'] for w in l] for l in sentences]


(sentences_train, tags_train) = read_conll_format("train")
(sentences_dev, tags_dev)     = read_conll_format("dev")

print(sentences_train[2])
print(tags_train[2])

sentencesChar = sentences2char(sentences_train)

print(sentencesChar[2])

['-START-', 'Peter', 'Blackburn', '-END-']
['START', 'I-PER', 'I-PER', 'END']
[['start', '-', 'S', 'T', 'A', 'R', 'T', '-', 'end'], ['start', 'P', 'e', 't', 'e', 'r', 'end'], ['start', 'B', 'l', 'a', 'c', 'k', 'b', 'u', 'r', 'n', 'end'], ['start', '-', 'E', 'N', 'D', '-', 'end']]


In [6]:
#Read GloVe embeddings.
def read_GloVe(filename):
    embeddings = {}
    for line in open(filename).readlines():
        #print(line)
        fields = line.strip().split(" ")
        word = fields[0]
        embeddings[word] = [float(x) for x in fields[1:]]
    return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

print(GloVe["the"])
print("dimension of glove embedding:", len(GloVe["the"]))

[0.27204, -0.06203, -0.1884, 0.023225, -0.018158, 0.0067192, -0.13877, 0.17708, 0.17709, 2.5882, -0.35179, -0.17312, 0.43285, -0.10708, 0.15006, -0.19982, -0.19093, 1.1871, -0.16207, -0.23538, 0.003664, -0.19156, -0.085662, 0.039199, -0.066449, -0.04209, -0.19122, 0.011679, -0.37138, 0.21886, 0.0011423, 0.4319, -0.14205, 0.38059, 0.30654, 0.020167, -0.18316, -0.0065186, -0.0080549, -0.12063, 0.027507, 0.29839, -0.22896, -0.22882, 0.14671, -0.076301, -0.1268, -0.0066651, -0.052795, 0.14258, 0.1561, 0.05551, -0.16149, 0.09629, -0.076533, -0.049971, -0.010195, -0.047641, -0.16679, -0.2394, 0.0050141, -0.049175, 0.013338, 0.41923, -0.10104, 0.015111, -0.077706, -0.13471, 0.119, 0.10802, 0.21061, -0.051904, 0.18527, 0.17856, 0.041293, -0.014385, -0.082567, -0.035483, -0.076173, -0.045367, 0.089281, 0.33672, -0.22099, -0.0067275, 0.23983, -0.23147, -0.88592, 0.091297, -0.012123, 0.013233, -0.25799, -0.02972, 0.016754, 0.01369, 0.32377, 0.039546, 0.042114, -0.088243, 0.30318, 0.087747, 0.1634

## Mapping Tokens to Indices

As in the last project, we will need to convert words in the dataset to numeric indices, so they can be presented as input to a neural network.  Code to handle this for you with sample usage is provided below.

In [7]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in sentences_train for w in l])
charCounts = Counter([c for l in sentences_train for w in l for c in w])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])
charSingletons = set([w for (w,c) in charCounts.items() if c == 1])

#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in sentences_train for w in l] + list(GloVe.keys())))}
char2i = {w:i+2 for i,w in enumerate(set([c for l in sentencesChar for w in l for c in w]))}
i2word = {i:w for w,i in word2i.items()}
i2char = {i:w for w,i in char2i.items()}

vocab_size = max(word2i.values()) + 1
char_vocab_size = max(char2i.values()) + 1

#Tag dictionaries.
tag2i = {w:i for i,w in enumerate(set([t for l in tags_train for t in l]))}
i2tag = {i:t for t,i in tag2i.items()}

#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
    if train and (w in singletons and random.random() > 0.5):
        return 1
    else:
        return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
    #1.0 => UNK
    return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

#Map a list of sentences containing to indices (character indices)
def sentences2indicesChar(chars, dictionary):
    #1.0 => UNK
    return [[[dictionary.get(c,1) for c in w] for w in l] for l in chars]

#Indices
X       = sentences2indices(sentences_train, word2i, train=True)
X_char  = sentences2indicesChar(sentencesChar, char2i)
Y       = sentences2indices(tags_train, tag2i)

print("vocab size:", vocab_size)
print("char vocab size:", char_vocab_size)
print()

print("index of word 'the':", word2i["the"])
print("word of index 253:", i2word[253])
print()

#Print out some examples of what the dev inputs will look like
for i in range(10):
    print(" ".join([i2word.get(w,'UNK') for w in X[i]]))

vocab size: 29148
char vocab size: 88

index of word 'the': 10450
word of index 253: court

-START- -DOCSTART- -END-
-START- EU rejects German call to boycott British lamb . -END-
-START- Peter Blackburn -END-
-START- BRUSSELS 1996-08-22 -END-
-START- The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . -END-
-START- Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . -END-
-START- " We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . -END-
-START- He said further scientific study was required and if it was found that action was needed it should be taken by the European Union . -END-

## Padding and Batching

In this assignment, you should train your models using minibatched SGD, rather than using a batch size of 1 as we did in the previous project.  When presenting multiple sentences to the network at the same time, we will need to pad them to be of the same length. We use [torch.nn.utils.rnn.pad_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html) to do so.

Below we provide some code to prepare batches of data to present to the network.  pad the sequence so that all the sequences have the same length.

**Side Note:** PyTorch includes utilities in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) to help with padding, batching, shuffling and some other things, but for this assignment we will do everything from scratch to help you see exactly how this works.

In [8]:
#Pad inputs to max sequence length (for batching)
def prepare_input(X_list):
    X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor) # padding the sequences with 0
    X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor) # consisting of 0 and 1, 0 for padded positions, 1 for non-padded positions
    return (X_padded, X_mask)

#Maximum word length (for character representations)
MAX_CLEN=32

def prepare_input_char(X_list):
    MAX_SLEN = max([len(l) for l in X_list])
    X_padded  = [l + [[]]*(MAX_SLEN-len(l))  for l in X_list]
    X_padded  = [[w[0:MAX_CLEN] for w in l] for l in X_padded]
    X_padded  = [[w + [1]*(MAX_CLEN-len(w)) for w in l] for l in X_padded]
    return torch.as_tensor(X_padded).type(torch.LongTensor)

#Pad outputs using one-hot encoding
def prepare_output_onehot(Y_list, NUM_TAGS=max(tag2i.values())+1):
    Y_onehot = [torch.zeros(len(l), NUM_TAGS) for l in Y_list]
    for i in range(len(Y_list)):
        for j in range(len(Y_list[i])):
            Y_onehot[i][j,Y_list[i][j]] = 1.0
    Y_padded = torch.nn.utils.rnn.pad_sequence(Y_onehot, batch_first=True).type(torch.FloatTensor)
    return Y_padded

print("max slen:", max([len(x) for x in X_char]))  #Max sequence length in the training data is 39.

(X_padded, X_mask) = prepare_input(X)
X_padded_char      = prepare_input_char(X_char)
Y_onehot           = prepare_output_onehot(Y)

print("X_padded:", X_padded.shape)
print("X_mask:", X_mask.shape)
print("X_padded_char:", X_padded_char.shape)
print("Y_onehot:", Y_onehot.shape)

max slen: 115
X_padded: torch.Size([14987, 115])
X_mask: torch.Size([14987, 115])
X_padded_char: torch.Size([14987, 115, 32])
Y_onehot: torch.Size([14987, 115, 10])


## **Your code starts here:** Basic LSTM Tagger (10 points)

OK, now you should have everything you need to get started.

Recall that your goal is to to implement the BiLSTM-CNN-CRF, as described in [(Ma and Hovy, 2016)](https://www.aclweb.org/anthology/P16-1101.pdf).  This is a relatively complex network with various components.  Below we provide starter code to break down your implementation into increasingly complex versions of the final model, starting with a Basic LSTM tagger.  This way you can be confident that each part is working correctly before incrementally increasing the complexity of your implementation.  This is generally a good approach to take when implementing complex models, since buggy PyTorch code is often partially working, but produces worse results than a correct implementation, so it's hard to know whether added complexities are helping or hurting.  Also, if you aren't able to match published results it's hard to know which component of your model has the problem (or even whether or not it is a problem in the published result!)

Fill in the functions marked as `TODO` in the code block below.  If everything is working correctly, you should be able to achieve an **F1 score of 0.87 on the dev set and 0.83 on the test set (with GloVe embeddings)**. You are required to initialize word embeddings with GloVe later, but you can randomly initialize the word embeddings in the beginning.

In [9]:
class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax
        self.embedding_layer = nn.Embedding(vocab_size, self.DIM_EMB)
        self.init_glove(GloVe)
        self.bilstm = nn.LSTM(self.DIM_EMB, DIM_HID, num_layers=1, bias=True, batch_first=True, bidirectional=True)
        self.linear_layer = nn.Linear(2 * DIM_HID, self.NUM_TAGS)
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, X, train=False):
        #TODO: Implement the forward computation.
        word_embeddings = self.embedding_layer(X)
        lstm_output, (final_h_state, final_c_state) = self.bilstm(word_embeddings)
        predicted_tags = self.linear_layer(lstm_output)
        log_probabilities = self.log_softmax(predicted_tags)
        return log_probabilities

    def init_glove(self, GloVe):
        #TODO: initialize word embeddings using GloVe (you can skip this part in your first version, if you want, see instructions below).
        
        # Create the weight matrix
        weight_matrix = torch.zeros((vocab_size, self.DIM_EMB))
        for word, i in word2i.items():
            if word in GloVe.keys(): 
                weight_matrix[i, :] = torch.FloatTensor(GloVe[word])
            else:
                weight_matrix[i, :] = torch.normal(mean=0.0, std=1.0, size=(self.DIM_EMB, ))

        # Load weight matrix into Embedding layer
        assert weight_matrix.shape == (vocab_size, self.DIM_EMB)
        #self.embedding_layer.load_state_dict({'weight': torch.FloatTensor(weight_matrix)})
        self.embedding_layer.weight.data = weight_matrix
        #self.embedding_layer.weight.requires_grad = False
        print('Initialized using GloVe embeddings')

    def inference(self, sentences):
        X = prepare_input(sentences2indices(sentences, word2i))[0].cuda()
        pred = self.forward(X).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

    def write_predictions(self, sentences, outFile):
        fOut = open(outFile, 'w')
        for s in sentences:
            y = self.inference([s])[0]
            #print("\n".join(y[1:len(y)-1]))
            fOut.write("\n".join(y[1:len(y)-1]))  #Skip start and end tokens
            fOut.write("\n\n")

#The following code will initialize a model and test that your forward computation runs without errors.
lstm_test   = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm_test.forward(prepare_input(X[0:5])[0])
Y_onehot    = prepare_output_onehot(Y[0:5])

#Check the shape of the lstm_output and one-hot label tensors.
print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

Initialized using GloVe embeddings
lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [10]:
#Read in the data

(sentences_dev, tags_dev)     = read_conll_format('dev')
(sentences_train, tags_train) = read_conll_format('train')
(sentences_test, tags_test)   = read_conll_format('test')

# Train your Model (10 points)

Next, implement the function below to train your basic BiLSTM tagger.  See [torch.nn.lstm](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).  Make sure to save your predictions on the test set (`test_pred_lstm.txt`) for submission to GradeScope. Feel free to change number of epochs, optimizer, learning rate and batch size.

In [11]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random

def shuffle_sentences(sentences, tags):
    shuffled_sentences = []
    shuffled_tags      = []
    indices = list(range(len(sentences)))
    random.shuffle(indices)
    for i in indices:
        shuffled_sentences.append(sentences[i])
        shuffled_tags.append(tags[i])
    return (shuffled_sentences, shuffled_tags)

nEpochs = 10

def train_basic_lstm(sentences, tags, lstm):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
    optimizer = optim.Adadelta(lstm.parameters(), lr=2)
    #optimizer = optim.Adam(lstm.parameters(), lr=0.001)
    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0
        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: Impelement gradient update.
            X, mask = prepare_input(sentences2indices(sentences_shuffled[batch : (batch + batchSize)], word2i, train=True))
            Y_batch_onehot = prepare_output_onehot(sentences2indices(tags_shuffled[batch : (batch + batchSize)] , tag2i)).cuda()
            Y_batch_labels = torch.argmax(Y_batch_onehot, axis=-1)

            # Y_predicted = lstm.forward(X.cuda())
            # mask_tensor = torch.stack((mask, mask, mask, mask, mask, mask, mask, mask, mask, mask), dim=2).cuda()
            # Y_predicted = Y_predicted * mask_tensor
            
            # assert Y_predicted.shape == Y_batch_onehot.shape
            
            # loss_function = nn.CrossEntropyLoss(reduction='sum')
            # loss = loss_function(Y_predicted.view(Y_predicted.shape[0] * Y_predicted.shape[1], Y_predicted.shape[-1]), Y_batch_labels.view(Y_batch_labels.shape[0] * Y_batch_labels.shape[1]).cuda())

            log_probabilities = lstm.forward(X.cuda())
            assert log_probabilities.shape == Y_batch_onehot.shape

            loss = -torch.sum(torch.sum(Y_batch_onehot * log_probabilities, dim=-1) * mask.cuda())
            
            
            totalLoss += loss
            loss.backward()
            optimizer.step()
            
            
        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_basic_lstm(sentences_train, tags_train, lstm)

Initialized using GloVe embeddings


  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 30843.2421875
conlleval:
processed 51578 tokens with 5942 phrases; found: 5799 phrases; correct: 5226.
accuracy:  98.15%; precision:  90.12%; recall:  87.95%; FB1:  89.02
              LOC: precision:  92.96%; recall:  90.53%; FB1:  91.73  1789
             MISC: precision:  84.73%; recall:  80.04%; FB1:  82.32  871
              ORG: precision:  84.12%; recall:  82.55%; FB1:  83.33  1316
              PER: precision:  94.24%; recall:  93.27%; FB1:  93.75  1823

----------------------------
-START-/START/START But/O/O for/O/O the/O/O great/O/O majority/O/O of/O/O us/O/O ,/O/O Hong/I-LOC/I-LOC Kong/I-LOC/I-LOC is/O/O our/O/O home/O/O and/O/O Hong/I-LOC/I-LOC Kong/I-LOC/I-LOC 's/O/O future/O/O is/O/O our/O/O future/O/O ./O/O "/O/O -END-/END/END
Predicted:	 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
Gold:		 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'I

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 6485.07177734375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6061 phrases; correct: 5411.
accuracy:  98.47%; precision:  89.28%; recall:  91.06%; FB1:  90.16
              LOC: precision:  96.02%; recall:  91.83%; FB1:  93.88  1757
             MISC: precision:  84.42%; recall:  85.79%; FB1:  85.10  937
              ORG: precision:  80.89%; recall:  88.07%; FB1:  84.33  1460
              PER: precision:  91.87%; recall:  95.11%; FB1:  93.46  1907



...

loss on epoch 7 = 176.39573669433594
conlleval:
processed 51578 tokens with 5942 phrases; found: 6064 phrases; correct: 5460.
accuracy:  98.59%; precision:  90.04%; recall:  91.89%; FB1:  90.95
              LOC: precision:  95.08%; recall:  94.61%; FB1:  94.84  1828
             MISC: precision:  82.40%; recall:  86.33%; FB1:  84.32  966
              ORG: precision:  83.37%; recall:  88.96%; FB1:  86.08  1431
              PER: precision:  94.24%; recall:  94.08%; FB1:  94.16  1839



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 8 = 156.6241912841797
conlleval:
processed 51578 tokens with 5942 phrases; found: 5940 phrases; correct: 5462.
accuracy:  98.64%; precision:  91.95%; recall:  91.92%; FB1:  91.94
              LOC: precision:  94.99%; recall:  96.08%; FB1:  95.53  1858
             MISC: precision:  88.17%; recall:  85.68%; FB1:  86.91  896
              ORG: precision:  88.51%; recall:  86.20%; FB1:  87.34  1306
              PER: precision:  93.14%; recall:  95.06%; FB1:  94.09  1880



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 9 = 110.42894744873047
conlleval:
processed 51578 tokens with 5942 phrases; found: 5957 phrases; correct: 5476.
accuracy:  98.68%; precision:  91.93%; recall:  92.16%; FB1:  92.04
              LOC: precision:  95.68%; recall:  95.26%; FB1:  95.47  1829
             MISC: precision:  85.89%; recall:  85.14%; FB1:  85.51  914
              ORG: precision:  87.80%; recall:  87.99%; FB1:  87.90  1344
              PER: precision:  94.17%; recall:  95.60%; FB1:  94.88  1870



In [12]:
#Evaluation on test data
lstm.write_predictions(sentences_test, 'test_pred_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_lstm.txt | perl conlleval.pl -d "\t"

--2022-03-13 22:15:27--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.1’

conlleval.pl.1      100%[===================>]  12.46K  --.-KB/s    in 0s      

2022-03-13 22:15:27 (113 MB/s) - ‘conlleval.pl.1’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5648 phrases; correct: 4882.
accuracy:  97.53%; precision:  86.44%; recall:  86.44%; FB1:  86.44
              LOC: precision:  89.99%; recall:  89.51%; FB1:  89.75  1659
             MISC: precision:  72.65%; recall:  74.93%; FB1:  73.77  724
              ORG: precision:  83.38%; recall:  83.38%; FB1:  83.38  1661
              PER: precisi

## Initialization with GloVe Embeddings (5 points)

If you haven't already, implement the `init_glove()` method in `BasicLSTMtagger` above.

Rather than initializing word embeddings randomly, it is common to use learned word embeddings (GloVe or Word2Vec), as discussed in lecture.  To make this simpler, we have already pre-filtered [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings to only contain words in the vocabulary of the CoNLL NER dataset, and loaded them into a dictionary (`GloVe`) at the beginning of this notebook.



## Character Embeddings (10 points)

Now that you have your basic LSTM tagger working, the next step is to add a convolutional network that computes word embeddings from character representations of words.  See Figure 2 and Figure 3 in the [Ma and Hovy](https://www.aclweb.org/anthology/P16-1101.pdf) paper.  We have provided code in `sentences2input_tensors` to convert sentences into lists of word and character indices.  See also [nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) and [MaxPool1d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html).

Hint: The nn.Conv1d accepts input size $(N, C_{in}, L_{in})$, but we have input size $(N, \text{SLEN}, \text{CLEN}, \text{EMB_DIM})$. We can reshape and [permute](https://pytorch.org/docs/stable/generated/torch.permute.html) our input to satisfy the nn.Conv1d, and recover the dimensions later.

Make sure to save your predictions on the test set, for submission to GradeScope.  You should be able to achieve **90 F1 / 85 F1 on the dev/test sets**.

In [13]:
import torch.nn.functional as F


class CharLSTMtagger(BasicLSTMtagger): 
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10):
        super(CharLSTMtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID)
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: Initialize parameters.
        self.char_embedding_layer = nn.Embedding(char_vocab_size, DIM_CHAR_EMB)
        self.conv_layer = nn.Conv1d(in_channels=DIM_CHAR_EMB, out_channels=30, kernel_size=3)
        # use max along last axis
        self.bilstm = nn.LSTM(self.DIM_EMB + DIM_CHAR_EMB, DIM_HID, num_layers=1, bias=True, batch_first=True, bidirectional=True)

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.

        # use X to get word embeddings
        word_embeddings = self.embedding_layer(X)

        character_embeddings = self.char_embedding_layer(X_char)
        shape = character_embeddings.shape
        conv_inputs = torch.permute(character_embeddings.view(shape[0] * shape[1], shape[2], shape[3]), (0, 2, 1))
        conv_outputs = self.conv_layer(conv_inputs)
        maxpool_outputs = conv_outputs.max(dim=-1)[0]  # max along L_out
        char_representations = maxpool_outputs.view(X_char.shape[0], X_char.shape[1], maxpool_outputs.shape[-1])

        # print(word_embeddings.shape)
        # print(character_embeddings.shape)
        # print(conv_inputs.shape)
        # print(conv_outputs.shape)
        # print(maxpool_outputs.shape)
        # print(char_representations.shape)

        concatenated_embeddings = torch.cat((word_embeddings, char_representations), dim=-1)
        #print(concatenated_embeddings.shape)

        lstm_output, (final_h_state, final_c_state) = self.bilstm(concatenated_embeddings)
        predicted_tags = self.linear_layer(lstm_output)
        log_probabilities = self.log_softmax(predicted_tags)

        return log_probabilities

    def sentences2input_tensors(self, sentences):
        (X, X_mask)   = prepare_input(sentences2indices(sentences, word2i))
        X_char        = prepare_input_char(sentences2indicesChar(sentences, char2i))
        return (X, X_mask, X_char)

    def inference(self, sentences):
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        pred = self.forward(X.cuda(), X_char.cuda()).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

char_lstm_test = CharLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output    = char_lstm_test.forward(prepare_input(X[0:5])[0], prepare_input_char(X_char[0:5]))
Y_onehot       = prepare_output_onehot(Y[0:5])

print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

Initialized using GloVe embeddings
lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [14]:
#Training LSTM w/ character embeddings. Feel free to change number of epochs, optimizer, learning rate and batch size.

nEpochs = 10

def train_char_lstm(sentences, tags, lstm):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
    optimizer = optim.Adadelta(lstm.parameters(), lr=2)
    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0

        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: Gradient update
            X, X_mask = prepare_input(sentences2indices(sentences_shuffled[batch : (batch + batchSize)], word2i, train=True))
            X_char = prepare_input_char(sentences2indicesChar(sentences_shuffled[batch : (batch + batchSize)], char2i))
            Y_batch_onehot = prepare_output_onehot(sentences2indices(tags_shuffled[batch : (batch + batchSize)] , tag2i)).cuda()

            log_probabilities = lstm.forward(X.cuda(), X_char.cuda())
            assert log_probabilities.shape == Y_batch_onehot.shape

            loss = -torch.sum(torch.sum(Y_batch_onehot * log_probabilities, dim=-1) * X_mask.cuda())
            
            totalLoss += loss
            loss.backward()
            optimizer.step()

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

char_lstm = CharLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_char_lstm(sentences_train, tags_train, char_lstm)

Initialized using GloVe embeddings


  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 27236.3359375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6044 phrases; correct: 5448.
accuracy:  98.61%; precision:  90.14%; recall:  91.69%; FB1:  90.91
              LOC: precision:  92.38%; recall:  95.65%; FB1:  93.98  1902
             MISC: precision:  86.65%; recall:  83.08%; FB1:  84.83  884
              ORG: precision:  83.60%; recall:  86.28%; FB1:  84.92  1384
              PER: precision:  94.34%; recall:  95.98%; FB1:  95.16  1874

----------------------------
-START-/START/START I/O/O think/O/O he/O/O got/O/O a/O/O little/O/O tight/O/O at/O/O a/O/O couple/O/O of/O/O moments/O/O ,/O/O "/O/O said/O/O Washington/I-LOC/I-PER ./O/O "/O/O -END-/END/END
Predicted:	 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'END']
Gold:		 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'O', 'END']
----------------------------
-START-/START/START For/

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 5851.30908203125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6019 phrases; correct: 5489.
accuracy:  98.73%; precision:  91.19%; recall:  92.38%; FB1:  91.78
              LOC: precision:  96.13%; recall:  93.41%; FB1:  94.75  1785
             MISC: precision:  85.82%; recall:  86.66%; FB1:  86.24  931
              ORG: precision:  85.89%; recall:  88.96%; FB1:  87.40  1389
              PER: precision:  93.05%; recall:  96.69%; FB1:  94.83  1914



...

loss on epoch 7 = 136.59896850585938
conlleval:
processed 51578 tokens with 5942 phrases; found: 6013 phrases; correct: 5542.
accuracy:  98.88%; precision:  92.17%; recall:  93.27%; FB1:  92.71
              LOC: precision:  95.44%; recall:  95.75%; FB1:  95.60  1843
             MISC: precision:  87.39%; recall:  87.96%; FB1:  87.68  928
              ORG: precision:  87.71%; recall:  87.84%; FB1:  87.78  1343
              PER: precision:  94.47%; recall:  97.39%; FB1:  95.91  1899



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 8 = 105.40133666992188
conlleval:
processed 51578 tokens with 5942 phrases; found: 6005 phrases; correct: 5517.
accuracy:  98.83%; precision:  91.87%; recall:  92.85%; FB1:  92.36
              LOC: precision:  95.84%; recall:  95.21%; FB1:  95.52  1825
             MISC: precision:  87.65%; recall:  86.98%; FB1:  87.32  915
              ORG: precision:  84.86%; recall:  89.86%; FB1:  87.29  1420
              PER: precision:  95.45%; recall:  95.60%; FB1:  95.52  1845



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 9 = 61.918663024902344
conlleval:
processed 51578 tokens with 5942 phrases; found: 6014 phrases; correct: 5543.
accuracy:  98.88%; precision:  92.17%; recall:  93.29%; FB1:  92.72
              LOC: precision:  94.85%; recall:  96.24%; FB1:  95.54  1864
             MISC: precision:  88.39%; recall:  87.53%; FB1:  87.96  913
              ORG: precision:  86.60%; recall:  88.67%; FB1:  87.62  1373
              PER: precision:  95.44%; recall:  96.58%; FB1:  96.01  1864



In [15]:
#Evaluation on test set
char_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm.txt | perl conlleval.pl -d "\t"

--2022-03-13 22:18:03--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.2’

conlleval.pl.2      100%[===================>]  12.46K  --.-KB/s    in 0s      

2022-03-13 22:18:03 (119 MB/s) - ‘conlleval.pl.2’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5771 phrases; correct: 5010.
accuracy:  97.74%; precision:  86.81%; recall:  88.70%; FB1:  87.75
              LOC: precision:  89.42%; recall:  91.67%; FB1:  90.53  1710
             MISC: precision:  74.05%; recall:  78.06%; FB1:  76.01  740
              ORG: precision:  82.47%; recall:  86.09%; FB1:  84.24  1734
              PER: precisi

## Conditional Random Fields (15 points)

Now we are ready to add a CRF layer to the `CharacterLSTMTagger`.  To train the model, implement `conditional_log_likelihood`, using the score (unnormalized log probability) of the gold sequence, in addition to the partition function, $Z(X)$, which is computed using the forward algorithm.  Then, you can simply use Pytorch's automatic differentiation to compute gradients by running backpropagation through the computation graph of the dynamic program (this should be very simple, so long as you are able to correctly implement the forward algorithm using a computation graph that is supported by PyTorch).  This approach to computing gradients for CRFs is discussed in Section 7.5.3 of the [Eisenstein Book](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)

You will also need to implement the Viterbi algorithm for inference during decoding.

After including CRF training and Viterbi decoding, you should be getting about **92 F1 / 88 F1 on the dev and test set**, respectively.

**IMPORTANT:** Note that training will be substantially slower this time - depending on the efficiency of your implementation, it could take about 5 minutes per epoch (e.g. 50 minutes for 10 iterations).  It is recommended to start out training on a single batch of data (and testing on this same batch), so that you can quickly debug, making sure your model can memorize the labels on a single batch, and then optimize your code.  Once you are fairly confident your code is working properly, then you can train using the full dataset.  We have provided a (commented out) line of code to switch between training on a single batch and the full dataset below.

**Hint #1:** While debugging your implementation of the Forward algorithm it is helpful to look at the loss during training.  The loss should never be less than zero (the log-likelihood should always be negative).

**Hint #2:** To sum log-probabilities in a numerically stable way at the end of the Forward algorithm, you will want to use [`torch.logsumexp`](https://pytorch.org/docs/stable/generated/torch.logsumexp.html).

In [16]:
#For F.max_pool1d()
import torch.nn.functional as F

class LSTM_CRFtagger(CharLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10, N_TAGS=max(tag2i.values())+1):
        super(LSTM_CRFtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, DIM_CHAR_EMB=DIM_CHAR_EMB)

        #TODO: Initialize parameters.

        self.transitionWeights = nn.Parameter(torch.zeros((N_TAGS, N_TAGS), requires_grad=True))
        nn.init.normal_(self.transitionWeights)
        self.N_TAGS = N_TAGS

    def gold_score(self, lstm_scores, Y, sLen):   # ADDED SLEN PARAMETER, SWITCH TO self.transitionWeights
        #TODO: compute score of gold sequence Y (unnormalized conditional log-probability)

        transition_scores = torch.zeros(lstm_scores.shape[0], 1).cuda() #(N,1)
        emission_scores = torch.zeros(lstm_scores.shape[0], 1).cuda()

        for i in range(lstm_scores.shape[0]): 
          # iterating over each sentence in a batch
          lstm_score = lstm_scores[i]

          for t in range(1, sLen[i]):
            transition_scores[i,0] += self.transitionWeights[Y[i, t-1], Y[i, t]]
          
          for t in range(sLen[i]):
            emission_scores[i,0] += lstm_score[t, Y[i, t]]
          
        score = transition_scores + emission_scores
        return score
        """
        score = torch.zeros(1)
        for t, lstm_hidden_t in enumerate(lstm_scores):
          score += self.transitionWeights[Y[t+1], Y[t]] + lstm_hidden_t[Y[t+1]]
        
        return score
        """

    #Forward algorithm for a single sentence
    #Efficiency will eventually be important here.  We recommend you start by 
    #training on a single batch and make sure your code can memorize the 
    #training data.  Then you can go back and re-write the inner loop using 
    #tensor operations to speed things up.
    def forward_algorithm(self, lstm_scores, sLen):
        #TODO: implement forward algorithm.
        """
        initial_alphas = lstm_scores[0]
        forward_probabilities = initial_alphas
        print(forward_probabilities)
        """

        initial_alphas = torch.clone(lstm_scores[:,0,:].view(lstm_scores.shape[0], lstm_scores.shape[2]))
        #initial_alphas = torch.zeros((lstm_scores.shape[0], lstm_scores.shape[2]))
        forward_probabilities = initial_alphas
        #print('Init Forward', forward_probabilities)
        for i in range(lstm_scores.shape[0]):
          # iterating over the sentences in the batch
          lstm_score = lstm_scores[i]
          for t in range(1, sLen[i]):
            # iterating over the time steps
            alphas_t = torch.zeros((self.N_TAGS, 1))
            emission_scores = lstm_score[t, :].view(-1, 1).expand(-1, self.N_TAGS)  # Ntags, Ntags 
            transition_scores = torch.transpose(self.transitionWeights[:,:], 0, 1).view(self.N_TAGS, self.N_TAGS)

            next_tags_var = forward_probabilities[i,:].view(1, -1).expand(self.N_TAGS, -1) + emission_scores + transition_scores
            #print('Next Tag Var', next_tags_var)
            alphas_t = torch.logsumexp(next_tags_var, dim=1)
            #print('Alphas t', alphas_t)
            forward_probabilities[i,:] = alphas_t.view(1, -1)
            #print('Forward Prob', forward_probabilities)
            
            # alphas_t = []
            # for next_tag in range(self.N_TAGS):
            #   emission_score = lstm_score[t, next_tag].view(1, -1).expand(1, self.N_TAGS)
            #   transition_score = self.transitionWeights[:, next_tag].view(1, -1)
            #   next_tag_var = forward_probabilities[i,:].cuda() + emission_score.cuda() + transition_score.cuda()

            #   alphas_t.append(torch.logsumexp(next_tag_var, dim=1).view(1))
            
            # forward_probabilities[i,:] = torch.cat(alphas_t).view(1, -1)
            
        #print(forward_probabilities, torch.logsumexp(forward_probabilities, dim=0), torch.logsumexp(forward_probabilities, dim=1))
        alpha = torch.logsumexp(forward_probabilities, dim=1)
        return alpha
        # initial_alphas = torch.full((1, self.N_TAGS), -10000.)

        # forward_probabilities = initial_alphas
        # print(forward_probabilities)
        # for lstm_scores_t in lstm_scores:1
        #   alphas_t = []
        #   for next_tag in range(self.N_TAGS):
        #     emission_score = lstm_scores_t[next_tag].view(1, -1).expand(1, self.N_TAGS)
        #     transition_score = self.transitionWeights[next_tag, :].view(1, -1)
        #     next_tag_var = forward_probabilities.cuda() + emission_score.cuda() + transition_score.cuda()
        #     print(next_tag_var)
        #     alphas_t.append(torch.logsumexp(next_tag_var).view(1))
        #   forward_probabilities = torch.cat(alphas_t).view(1, -1)
        
        # return alpha

    def conditional_log_likelihood(self, sentences, tags, train=False):
        #Todo: compute conditional log likelihood of Y (use forward_algorithm and gold_score)
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        Y = torch.argmax(prepare_output_onehot(sentences2indices(tags, tag2i)), dim=-1).cuda()

        lstm_scores = self.forward(X.cuda(), X_char.cuda(), train=train)

        forward_scores = self.forward_algorithm(lstm_scores, torch.count_nonzero(X_mask, dim=1))
        gold_scores = self.gold_score(lstm_scores, Y, torch.count_nonzero(X_mask, dim=1))

        return torch.sum(forward_scores - gold_scores) # sum over batch

    def viterbi(self, lstm_scores, sLen):   # works on individual sentences
        #TODO: Implement Viterbi algorithm, soring backpointers to recover the argmax sequence.  Returns the argmax sequence in addition to its unnormalized conditional log-likelihood.
        
        backpointers = []
        #initial_viterbi_variables = torch.full((1, self.N_TAGS), -10000.)
        initial_viterbi_variables = torch.clone(lstm_scores[0,:])
        viterbi_variables = initial_viterbi_variables
        #print('init shape: ', viterbi_variables.shape)

        #for lstm_scores_t in lstm_scores:
        for t in range(1, sLen): #lstm_scores.shape[0]):   # might have to switch to sLen # start from 1 or 0?
          backpointers_t = []
          viterbi_variables_t = []

          for next_tag in range(self.N_TAGS):
            # no need to add emissions to compute max (it's independent of the previous tag)
            next_tag_var = (viterbi_variables + self.transitionWeights[:, next_tag].view(viterbi_variables.shape))
            #print('nexttagvar shape: ',next_tag_var.shape)
            best_tag_index = torch.argmax(next_tag_var) # specify dimension?
            #print('best tag', best_tag_index)
            backpointers_t.append(best_tag_index)
            #print(next_tag_var, best_tag_index)
            viterbi_variables_t.append(next_tag_var[best_tag_index].view(1))
          # add emissions
          viterbi_variables = (torch.cat(viterbi_variables_t).view(viterbi_variables.shape) + lstm_scores[t, :].view(viterbi_variables.shape))
          backpointers.append(backpointers_t)

        best_tag_index = torch.argmax(viterbi_variables)
        best_score = viterbi_variables[best_tag_index]
        best_path = [best_tag_index]
        for backpointers_t in reversed(backpointers): # index from [1:]?
          best_tag_index = backpointers_t[best_tag_index]
          best_path.append(best_tag_index)
        best_path.reverse()
        #print('Best Path:', best_path)
        #print('Best Score:', best_score)
        return best_path, best_score

    #Computes Viterbi sequences on a batch of data.
    def viterbi_batch(self, sentences):
        viterbiSeqs = []
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        lstm_scores = self.forward(X.cuda(), X_char.cuda())
        for s in range(len(sentences)):
            (viterbiSeq, ll) = self.viterbi(lstm_scores[s], len(sentences[s]))
            viterbiSeqs.append(viterbiSeq)
        return viterbiSeqs

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        # use X to get word embeddings
        word_embeddings = self.embedding_layer(X)

        character_embeddings = self.char_embedding_layer(X_char)
        shape = character_embeddings.shape
        conv_inputs = torch.permute(character_embeddings.view(shape[0] * shape[1], shape[2], shape[3]), (0, 2, 1))
        conv_outputs = self.conv_layer(conv_inputs)
        maxpool_outputs = conv_outputs.max(dim=-1)[0]  # max along L_out
        char_representations = maxpool_outputs.view(X_char.shape[0], X_char.shape[1], maxpool_outputs.shape[-1])

        concatenated_embeddings = torch.cat((word_embeddings, char_representations), dim=-1)

        if train == True: # apply dropout to LSTM inputs
          concatenated_embeddings = nn.Dropout(0.5)(concatenated_embeddings)

        lstm_output, (final_h_state, final_c_state) = self.bilstm(concatenated_embeddings)

        if train == True: # apply dropout to LSTM outputs
          lstm_output = nn.Dropout(0.5)(lstm_output)
        predicted_tags = self.linear_layer(lstm_output)
        
        
        return predicted_tags

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", [Y_pred[i][j] for j in range(len(words[i]))])
            print("Gold:\t\t", tags[i])

    #Need to use Viterbi this time.
    def inference(self, sentences, viterbi=True):
        pred = self.viterbi_batch(sentences)
        return [[i2tag[pred[i][j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

lstm_crf = LSTM_CRFtagger(DIM_EMB=300).cuda()

Initialized using GloVe embeddings


In [17]:
print(lstm_crf.conditional_log_likelihood(sentences_dev[0:3], tags_dev[0:3]))

tensor(198.4649, device='cuda:0', grad_fn=<SumBackward0>)


In [18]:
#CharLSTM-CRF Training. Feel free to change number of epochs, optimizer, learning rate and batch size.
import tqdm
import os
import subprocess
import random

nEpochs = 10

#Get CoNLL evaluation script
os.system('wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl')

def train_crf_lstm(sentences, tags, lstm):
    #optimizer = optim.Adadelta(lstm.parameters(), lr=1.0)
    #TODO: initialize optimizer
    optimizer = optim.Adadelta(lstm.parameters(), lr=2)
    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0
        lstm.train()

        #Shuffle the sentences
        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: take gradient step on a batch of data.
            
            sentences_batch = sentences_shuffled[batch : batch + batchSize]
            tags_batch = tags_shuffled[batch : batch + batchSize]

            loss = lstm.conditional_log_likelihood(sentences_batch, tags_batch, train=True)
            
            totalLoss += loss / batchSize
            loss.backward()
            optimizer.step()

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            lstm.eval()
            s = random.sample(range(50), 5)
            lstm.print_predictions([sentences_train[i] for i in s], [tags_train[i] for i in s])   #Print predictions on train data (useful for debugging)

crf_lstm = LSTM_CRFtagger(DIM_HID=500, DIM_EMB=300, DIM_CHAR_EMB=30).cuda()
train_crf_lstm(sentences_train, tags_train, crf_lstm)             #Train on the full dataset
#train_crf_lstm(sentences_train[0:50], tags_train[0:50], crf_lstm)          #Train only the first batch (use this during development/debugging)

Initialized using GloVe embeddings


  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 40881.1875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6038 phrases; correct: 5406.
accuracy:  98.38%; precision:  89.53%; recall:  90.98%; FB1:  90.25
              LOC: precision:  95.30%; recall:  93.85%; FB1:  94.57  1809
             MISC: precision:  88.56%; recall:  81.45%; FB1:  84.86  848
              ORG: precision:  77.69%; recall:  86.73%; FB1:  81.96  1497
              PER: precision:  93.84%; recall:  95.98%; FB1:  94.90  1884

----------------------------
-START-/START/START German/I-MISC/I-MISC July/O/O car/O/O registrations/O/O up/O/O 14.2/O/O pct/O/O yr/O/O //O/O yr/O/O ./O/O -END-/END/END
Predicted:	 ['START', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
Gold:		 ['START', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
----------------------------
-START-/START/START FRANKFURT/I-LOC/I-LOC 1996-08-22/O/O -END-/END/END
Predicted:	 ['START', 'I-LOC', 'O', 'END']
Gold:		 ['START', 'I-LOC', 'O', 

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 10863.5283203125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5911 phrases; correct: 5498.
accuracy:  98.75%; precision:  93.01%; recall:  92.53%; FB1:  92.77
              LOC: precision:  96.08%; recall:  94.61%; FB1:  95.34  1809
             MISC: precision:  87.29%; recall:  86.44%; FB1:  86.87  913
              ORG: precision:  88.29%; recall:  88.81%; FB1:  88.55  1349
              PER: precision:  96.30%; recall:  96.20%; FB1:  96.25  1840



...

loss on epoch 7 = 2738.097900390625
conlleval:
processed 51578 tokens with 5942 phrases; found: 5914 phrases; correct: 5575.
accuracy:  98.99%; precision:  94.27%; recall:  93.82%; FB1:  94.05
              LOC: precision:  96.91%; recall:  95.54%; FB1:  96.22  1811
             MISC: precision:  87.53%; recall:  88.29%; FB1:  87.90  930
              ORG: precision:  92.27%; recall:  91.72%; FB1:  92.00  1333
              PER: precision:  96.52%; recall:  96.42%; FB1:  96.47  1840



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 8 = 2279.11328125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5910 phrases; correct: 5554.
accuracy:  98.96%; precision:  93.98%; recall:  93.47%; FB1:  93.72
              LOC: precision:  95.32%; recall:  96.46%; FB1:  95.89  1859
             MISC: precision:  90.27%; recall:  86.55%; FB1:  88.37  884
              ORG: precision:  90.74%; recall:  90.60%; FB1:  90.67  1339
              PER: precision:  96.77%; recall:  96.04%; FB1:  96.40  1828



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 9 = 2182.818359375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5936 phrases; correct: 5575.
accuracy:  98.95%; precision:  93.92%; recall:  93.82%; FB1:  93.87
              LOC: precision:  95.72%; recall:  96.19%; FB1:  95.95  1846
             MISC: precision:  87.97%; recall:  89.59%; FB1:  88.77  939
              ORG: precision:  91.66%; recall:  90.98%; FB1:  91.32  1331
              PER: precision:  96.81%; recall:  95.66%; FB1:  96.23  1820



In [19]:
crf_lstm.eval()
crf_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm_crf.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm_crf.txt | perl conlleval.pl -d "\t"

--2022-03-13 23:11:36--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.4’

conlleval.pl.4      100%[===================>]  12.46K  --.-KB/s    in 0s      

2022-03-13 23:11:36 (110 MB/s) - ‘conlleval.pl.4’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5635 phrases; correct: 5052.
accuracy:  97.93%; precision:  89.65%; recall:  89.45%; FB1:  89.55
              LOC: precision:  90.79%; recall:  93.41%; FB1:  92.08  1716
             MISC: precision:  77.35%; recall:  82.19%; FB1:  79.70  746
              ORG: precision:  87.76%; recall:  86.33%; FB1:  87.04  1634
              PER: precisi

In [39]:
def test_gold():
    import math
    lstm_crf = LSTM_CRFtagger(DIM_EMB=300, N_TAGS=3).cuda()
    lstm_scores = torch.tensor([
        [
            [1., 2, 3], # word 1
            [4, 5, 6],# word 2
            [1245, 1234, 6456] #padded 
        ],
        [
            [1., 2, 3], # word 1
            [4, 5, 6], # word 2
            [1, 2, 3] # word 3
        ],
    ])
    Y = torch.tensor([
        [0, 2, 1],
        [0, 2, 1]
    ])
    print(lstm_scores.shape)
    lstm_crf.transitionWeights2 = torch.tensor([
            [1., 1, 1],
            [2, 2, 2],
            [3, 3, 3]
    ]).cuda()
    ans = lstm_crf.gold_score(lstm_scores.cuda(), Y.cuda(), torch.tensor([2, 3]).cuda() )
    print("gold path score: ", ans)
    assert(math.isclose(ans[0], 8, abs_tol=0.1))
    assert(math.isclose(ans[1], 13, abs_tol=0.1))

test_gold()

Initialized using GloVe embeddings
torch.Size([2, 3, 3])
gold path score:  tensor([[ 8.],
        [13.]], device='cuda:0')


In [67]:
def test_forward():
    import math
    lstm_crf = LSTM_CRFtagger(DIM_EMB=300, N_TAGS=3).cuda()
    lstm_scores = torch.tensor([
        [
            [1., 2, 3], # word 1
            [4, 5, 6],# word 2
            [1245, 1234, 6456]
        ],
        [
            [1., 2, 3], # word 1
            [4, 5, 6], # word 2
            [1, 2, 3] # word 3
        ],
    ])
    print(lstm_scores.shape)
    lstm_crf.transitionWeights2 = torch.tensor([
            [1., 1, 1],
            [2, 2, 2],
            [3, 3, 3]
    ]).cuda()
    ans = lstm_crf.forward_algorithm(lstm_scores.cuda(), torch.tensor([2, 3]).cuda() )
    print("total path score: ", ans)
    assert(math.isclose(ans[0], 12.5, abs_tol=0.1))
    assert(math.isclose(ans[1], 18.6, abs_tol=0.1))


test_forward()

Initialized using GloVe embeddings
torch.Size([2, 3, 3])
total path score:  tensor([12.5505, 18.6935], device='cuda:0')


In [37]:
def test_viterbi():
    import math
    lstm_crf = LSTM_CRFtagger(DIM_EMB=300, N_TAGS=3).cuda()
    lstm_scores = torch.tensor([
        [1., 2, 3], # word 1
        [4, 5, 6], # word 2
        # [1, 2, 3] # word 3
    ])

    print(lstm_scores.shape)
    lstm_crf.transitionWeights2 = torch.tensor([
            [1., 20, 1],
            [2, 2, 2],
            [3, 3, 3]
    ]).cuda()
    ans = lstm_crf.viterbi(lstm_scores.cuda(), 2)[0]
    print("specified path score: ", ans)
    assert(math.isclose(ans[0], 0, abs_tol=0.1))
    assert(math.isclose(ans[1], 1, abs_tol=0.1))

test_viterbi()

Initialized using GloVe embeddings
torch.Size([2, 3])
specified path score:  [tensor(0, device='cuda:0'), tensor(1, device='cuda:0')]


## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_lstm.txt
* test_pred_cnn_lstm.txt
* test_pred_cnn_lstm_crf.txt
* NER_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.
